<a href="https://colab.research.google.com/github/trailblazerakash/MachineLearning/blob/master/hdfc/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np
%matplotlib inline

from itertools import chain
from sklearn import preprocessing, ensemble
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split




In [30]:

train_df = pd.read_csv('/content/drive/My Drive/HDFC/d407ebe0db9a11e9/DataSet/Train.csv')
test_df=pd.read_csv('/content/drive/My Drive/HDFC/d407ebe0db9a11e9/DataSet/Test.csv')


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (746,835) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (700,731,740,752,761,789,811,820,829,841,850) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
df_dtypes = pd.DataFrame(train_df.dtypes,columns=['dtypes'])
df_dtypes['first_value'] = train_df.loc[0].values


In [0]:
df_dtypes['Null_Count'] = train_df[train_df.columns].isnull().sum()
   

In [33]:
df_dtypes.head()

,dtypes,first_value,Null_Count
Col1,object,RIGHGSN,0
Col2,int64,1,0
Col3,float64,10867.1,0
Col4,float64,8648.34,0
Col5,float64,15766.2,0


In [0]:
df.isna().mean()

In [0]:
tdf = train_df.loc[:, train_df.isna().mean() < .25]
test_df_itm=test_df.loc[:, test_df.isna().mean() < .25]

In [36]:
test_df.shape

(20442, 2394)

In [0]:
#@title Default title text
#NOT WORKING PROPERLY

def get_redundant_pairs(df):
    '''Get diagonal and lower triangular pairs of correlation matrix'''
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

def get_top_abs_correlations(df, n=5):
    au_corr = df.corr().abs().unstack()
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    return au_corr[0:n]


In [0]:

print("Top Absolute Correlations")
print(get_top_abs_correlations(tdf, 100))

In [0]:
def get_columns_count(df):
  dtype_df = df.dtypes.reset_index()
  dtype_df.columns = ["Count", "Column Type"]
  return dtype_df.groupby("Column Type").aggregate('count').reset_index()

In [0]:
get_columns_count(tdf)

In [0]:
get_columns_count(test_df)

In [0]:
dtype_df = tdf.dtypes.reset_index()
dtype_df.columns = ["Count", "Column Type"]
dtype_df.groupby("Column Type").aggregate('count').reset_index()


In [0]:
dtype_df.ix[:10,:]

In [0]:
unique_values_dict = {}
for col in tdf.columns:
    if col not in ["Col1","Col2"]:
        unique_value = str(np.sort(tdf[col].unique()).tolist())
        tlist = unique_values_dict.get(unique_value, [])
        tlist.append(col)
        unique_values_dict[unique_value] = tlist[:]
for unique_val, columns in unique_values_dict.items():
    print("Columns containing the unique values : ",unique_val)
    print(columns)
    print("--------------------------------------------------")

In [0]:
correlation_threshold = 0.99 #
train_integer = tdf.drop(["Col1","Col2"],axis=1)
cor = train_integer.corr()

In [38]:

cor.loc[:,:] =  np.tril(cor, k=-1)
cor = cor.stack()
correlated = cor[cor > correlation_threshold].reset_index().loc[:,['level_0','level_1']]
correlated = correlated.query('level_0 not in level_1')
correlated_array =  correlated.groupby('level_0').agg(lambda x: set(chain(x.level_0,x.level_1))).values
correlated_array

array([[{'Col1038', 'Col946'}],
       [{'Col1042', 'Col950'}],
       [{'Col982', 'Col1074'}],
       [{'Col1075', 'Col983'}],
       [{'Col94', 'Col108'}],
       [{'Col1080', 'Col988'}],
       [{'Col1081', 'Col989'}],
       [{'Col1088', 'Col996'}],
       [{'Col1089', 'Col997'}],
       [{'Col998', 'Col1090'}],
       [{'Col1076', 'Col1168', 'Col984'}],
       [{'Col993', 'Col1085', 'Col1177'}],
       [{'Col1086', 'Col1178', 'Col994'}],
       [{'Col1188', 'Col1096'}],
       [{'Col1268', 'Col1084', 'Col992', 'Col1176'}],
       [{'Col1196', 'Col1288', 'Col1104'}],
       [{'Col121', 'Col129'}],
       [{'Col1292', 'Col1200'}],
       [{'Col123', 'Col131'}],
       [{'Col1326', 'Col1006', 'Col1310'}],
       [{'Col125', 'Col133'}],
       [{'Col1334', 'Col1022'}],
       [{'Col942', 'Col1344'}],
       [{'Col962', 'Col1368', 'Col1352'}],
       [{'Col136', 'Col137'}],
       [{'Col1370', 'Col970'}],
       [{'Col1376', 'Col978'}],
       [{'Col1392', 'Col1302', 'Col1078', 'Col986

In [39]:
correlated_features = []
for sets in correlated_array:
    element_list = list(sets[0])
    for idx, el in enumerate(element_list):
        if idx is not 0:
            correlated_features.append(el)

print (correlated_features)


['Col946', 'Col950', 'Col1074', 'Col983', 'Col108', 'Col988', 'Col989', 'Col996', 'Col997', 'Col1090', 'Col1168', 'Col984', 'Col1085', 'Col1177', 'Col1178', 'Col994', 'Col1096', 'Col1084', 'Col992', 'Col1176', 'Col1288', 'Col1104', 'Col129', 'Col1200', 'Col131', 'Col1006', 'Col1310', 'Col133', 'Col1022', 'Col1344', 'Col1368', 'Col1352', 'Col137', 'Col970', 'Col978', 'Col1302', 'Col1078', 'Col986', 'Col1394', 'Col1400', 'Col1416', 'Col1418', 'Col1328', 'Col1114', 'Col1338', 'Col1342', 'Col1034', 'Col1458', 'Col1442', 'Col1460', 'Col1070', 'Col1482', 'Col1306', 'Col1486', 'Col1398', 'Col1488', 'Col1504', 'Col1506', 'Col1190', 'Col1508', 'Col1206', 'Col1430', 'Col1126', 'Col1548', 'Col1532', 'Col1550', 'Col1556', 'Col1572', 'Col1582', 'Col1406', 'Col1496', 'Col1266', 'Col1174', 'Col1590', 'Col990', 'Col1320', 'Col1082', 'Col1500', 'Col1592', 'Col1412', 'Col1322', 'Col1282', 'Col1580', 'Col1598', 'Col1330', 'Col1600', 'Col1420', 'Col1331', 'Col1601', 'Col1511', 'Col1298', 'Col160', 'Col161

In [0]:
non_cor_train_df = tdf.drop(correlated_features, axis=1 )

In [42]:
non_cor_train_df.shape


(17521, 1368)

In [0]:
non_cor_train_df.head()

In [0]:
import seaborn as sns; sns.set(style="ticks", color_codes=True)
g = sns.pairplot(non_cor_train_df)

In [43]:
unique_values_list = non_cor_train_df.nunique()
unique_values_list=unique_values_list.reset_index()
unique_values_list.columns = ['Col',"Count"]
unique_values_list['Col'].count()

1368

In [46]:
continious =  unique_values_list[unique_values_list['Count'] > 990]
cont_cols = continious['Col']
cont_df = non_cor_train_df[cont_cols]
cat_df = non_cor_train_df.drop(cont_cols,axis=1)
cat_df.head()

,Col2,Col29,Col30,Col31,Col32,Col33,Col34,Col35,Col36,Col44,Col45,Col46,Col47,Col48,Col49,Col50,Col51,Col52,Col53,Col54,Col55,Col56,Col57,Col58,Col59,Col60,Col61,Col62,Col63,Col64,Col65,Col66,Col67,Col68,Col69,Col70,Col71,Col72,Col73,Col74,...,Col2331,Col2333,Col2334,Col2335,Col2336,Col2337,Col2338,Col2340,Col2342,Col2343,Col2346,Col2347,Col2348,Col2349,Col2350,Col2355,Col2356,Col2357,Col2358,Col2360,Col2362,Col2365,Col2366,Col2367,Col2369,Col2370,Col2371,Col2375,Col2377,Col2378,Col2381,Col2382,Col2383,Col2384,Col2385,Col2388,Col2389,Col2391,Col2393,Col2396
0,1,1,1,1,1,1,1,0,6,2,3,2,3,2,2,2,2,1,3,1,3,1,2,1,2,1,1,1,1,1,1,1,1,3,1,3,1,3,1,3,...,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,214.0,22.0
1,0,1,1,3,4,4,0,4,5,3,0,2,0,2,0,1,0,3,0,2,0,2,0,1,0,2,0,2,0,2,0,1,0,2,0,2,0,2,0,2,...,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,41.0,12.0
2,0,3,3,4,12,4,0,2,4,2,1,2,0,1,0,1,0,2,1,2,0,1,0,1,0,1,1,1,0,1,0,1,0,2,1,2,1,1,0,1,...,0,0,2,0,2,0,1,0,0,0,1,0,2,1,0,0,0,0,1,1,1,0,0,0,0,1,1,0,0,2,0,0,0,0,0,0,1,0,378.0,21.0
3,0,2,2,2,24,1,0,1,4,2,1,2,1,1,1,1,1,2,1,2,1,1,1,1,1,2,0,2,0,1,0,1,0,2,1,2,1,2,1,2,...,0,0,2,0,3,0,1,0,1,2,0,1,0,1,1,0,0,0,0,0,0,0,0,1,0,1,2,0,1,2,0,0,0,1,2,0,0,0,702.0,951.0
4,0,4,3,4,68,8,0,2,4,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,0,2,0,2,0,2,0,2,1,2,1,2,1,2,1,...,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0,0,1305.0,2205.0


In [47]:
wrongly_classified =['Col2393','Col2396']
for col in wrongly_classified:
  cont_df[col] = non_cor_train_df[col]
  cat_df.drop(col,axis=1,inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
cat_df = cat_df.drop('Col2', axis=1)

In [0]:
def get_missing_count(df):
  missing_df = df.isnull().sum(axis=0).reset_index()
  missing_df.columns = ['column_name', 'missing_count']
  missing_df = missing_df.loc[missing_df['missing_count']>0]
  return missing_df

In [0]:
def get_non_missing_columns(df):
  missing_df = df.isnull().sum(axis=0).reset_index()
  missing_df.columns = ['column_name', 'missing_count']
  missing_df = missing_df.loc[missing_df['missing_count']==0]
  return missing_df

In [0]:
cat_non_miss_columns =list(get_non_missing_columns(cat_df)['column_name'])
cont_non_miss_columns =list(get_non_missing_columns(cont_df)['column_name'])



In [52]:
final_cols = cat_non_miss_columns+cont_non_miss_columns

training_df = non_cor_train_df[final_cols]
training_df['Col2'] = non_cor_train_df['Col2']


#Expliciltly choosing only already decided features
testing_df = test_df_itm[final_cols]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [55]:
get_missing_count(testing_df)

,column_name,missing_count


In [0]:
ids= testing_df.Col1
testing_df.drop('Col1',axis=1,inplace=True)

In [0]:
target=training_df.Col2
candidate_train_predictors = training_df.drop(['Col1','Col2'], axis=1)
low_cardinality_cols = [cname for cname in candidate_train_predictors.columns
                                       if candidate_train_predictors[cname].nunique()< 20 and
                                       candidate_train_predictors[cname].dtype=="object"]

numeric_cols = [cname for cname in candidate_train_predictors.columns
                               if candidate_train_predictors[cname].dtype in ['int64', 'float64']]
useful_cols = low_cardinality_cols + numeric_cols
train_predictors = candidate_train_predictors[useful_cols]


In [0]:
one_hot_encoded_data = pd.get_dummies(train_predictors)

In [0]:
x_train,x_test,y_train,y_test = train_test_split(one_hot_encoded_data,target,test_size = 0.3,random_state = 1)

In [64]:
from sklearn.tree import DecisionTreeClassifier
dtree = DecisionTreeClassifier()
dtree.fit(x_train,y_train)
prediction = dtree.predict(x_test)

print('With Decision tree accuracy is: ',dtree.score(x_test,y_test))

With Decision tree accuracy is:  0.8333650370933993


In [0]:
prediction=dtree.predict(testing_df)


In [0]:
dataset = pd.DataFrame({'Col1': list(ids), 'Col2': prediction}, columns=['Col1', 'Col2'])
dataset.to_csv('/content/drive/My Drive/HDFC/d407ebe0db9a11e9/DataSet/Submission.csv',header=True,index=False)

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
param_grid = {"C": np.logspace(-3,3,7), "penalty" : ["l1","l2"]}

logreg = LogisticRegression()
logreg_cv = GridSearchCV(logreg,param_grid,cv = 10)
logreg_cv.fit(x_train,y_train)  

In [0]:
print("tuned hyperparameters: (best parameters): ", logreg_cv.best_params_)
print("accuracy: ", logreg_cv.best_score_)



In [0]:
logreg2 = LogisticRegression(C = 10, penalty="l2")
logreg2.fit(x_train,y_train)
print("score: ", logreg2.score(x_test,y_test))

In [0]:
xgboost = XGBClassifier(max_depth=5, learning_rate=0.01, n_estimators=100, gamma=0, 
                        min_child_weight=1, subsample=0.8, colsample_bytree=0.8, reg_alpha=0.005)

xgboost.fit(X_train, y_train)
preds = xgboost.predict(X_test)

accuracy = (preds == y_test).sum().astype(float) / len(preds)*100

print("XGBoost's prediction accuracy WITH optimal hyperparameters is: %3.2f" % (accuracy))

In [0]:


print("F1-Score:", f1_score(sdss['class'], predictions, average='micro'))

